In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### EC3D dataset
For the analysis EC3D dataset was used. Here is it's overview:

| Exercise                   | Instruction label | Sub1 | Sub2 | Sub3 | Sub4 | Total |
|-----------------------------------|-------------------|------|------|------|------|-------|
| Squats                       | Correct           | 10   | 10   | 11   | 10   | 41 |
|                              | Feet too wide     | 5    | 8    | 5    | 5    | 23    |
|                              | Knees inward      | 6    | 7    | 5    | 5    | 23    |
|                              | Not low enough    | 5    | 7    | 5    | 4    | 21    |
|                              | Front bent        | 5    | 6    | 6    | 7    | 24    |
| Lunges                       | Correct              | 12   | 11   | 11   | 12   | 46    |
|                              | Not low enough    | 10   | 10   | 10   | 10   | 40    |
|                              | Knee passes toe   | 10   | 10   | 11   | 10   | 41    |
| Planks                       | Correct               | 7    | 8    | 11   | 7    | 33    |
|                              | Arched back       | 5    | 5    | 11   | 9    | 30    |
|                              | Hunch back        | 10   | 10   | 11   | 9    | 40    |


In [ ]:
LABELS_COLUMNS = ["exercise", "subject", "label", "rep", "frame"]

In [ ]:
dataset = pd.read_pickle("../../data/data_3D.pickle")

poses = dataset["poses"]

labels_df = pd.DataFrame(dataset["labels"], columns=LABELS_COLUMNS)
labels_df["frame"] = np.arange(len(labels_df))
labels_df[LABELS_COLUMNS[2:]] = labels_df[LABELS_COLUMNS[2:]].astype("int")

In [ ]:
x = labels_df[labels_df["exercise"] == "Plank"]
for _, person in x[x["label"] == 8].groupby("subject"):
    print(person["rep"].unique())

### Dataset processing

To prepare data for deep models we need to keep only label and positions in time for each exercise.


In [ ]:
def get_rep_frames_from_df(df: pd.DataFrame) -> pd.Grouper:
    groups = df.groupby("subject")

    return [
        rep["frame"].values
        for _, subject_group in groups
        for _, rep in subject_group.groupby("rep")
    ]


def get_df_from_frames(
    df: pd.DataFrame, exercises: np.ndarray, label: str
) -> pd.DataFrame:
    frames = get_rep_frames_from_df(df)
    final_reps_df = pd.DataFrame()
    for rep_num, frames_rep in enumerate(frames, start=1):
        rep_3d_joints = exercises[frames_rep]

        rep_3d_joints_x = rep_3d_joints[:, 0, :].reshape(-1)
        rep_3d_joints_y = rep_3d_joints[:, 1, :].reshape(-1)
        rep_3d_joints_z = rep_3d_joints[:, 2, :].reshape(-1)
        frames_num = len(rep_3d_joints_x) // 25
        final_rep = np.array(
            [
                np.full_like(rep_3d_joints_x, rep_num, dtype=int),
                rep_3d_joints_x,
                rep_3d_joints_y,
                rep_3d_joints_z,
                np.repeat(np.arange(frames_num, dtype=int), 25),
                np.full_like(rep_3d_joints_x, label, dtype="<U15"),
            ]
        ).T
        final_reps_df = pd.concat(
            [
                final_reps_df,
                pd.DataFrame(
                    final_rep, columns=["rep", "x", "y", "z", "frame", "label"]
                ),
            ],
            axis=0,
        )

    final_reps_df["rep"] = final_reps_df["rep"].astype("int")
    final_reps_df["frame"] = final_reps_df["frame"].astype("int")

    return final_reps_df

### Squat

In [ ]:
LABELS_COLUMNS = ["exercise", "subject", "label", "rep", "frame"]

SQUAT_LABELS = {
    1: "correct",
    2: "feet_too_wide",
    3: "knees_inwards",
    4: "not_low_enough",
    5: "front_bend",
}
LUNGES_LABELS = {
    1: "correct",
    4: "not_low_enough",
    6: "knee_passes_toe",
}
PLANK_LABELS = {
    1: "correct",
    7: "arched_back",
    8: "hunched_back",
}

In [ ]:
squat_df = labels_df[labels_df["exercise"] == "SQUAT"]
lunges_df = labels_df[labels_df["exercise"] == "Lunges"]
planks_df = labels_df[labels_df["exercise"] == "Plank"]

In [ ]:
squat_df

In [ ]:
import os

output_dir = "../../data"

for df, labels in [
    (squat_df, SQUAT_LABELS),
    (lunges_df, LUNGES_LABELS),
    (planks_df, PLANK_LABELS),
]:
    exercise_dfs = []
    for label, label_name in labels.items():
        exercise_dfs.append(
            get_df_from_frames(df[df["label"] == label], poses, label_name)
        )

    pd.concat(exercise_dfs).to_csv(
        os.path.join(output_dir, f"{df['exercise'].values[0].lower()}.csv"),
        index=False,
    )

### Visualization 

In [ ]:
openpose_connections = [
    (0, 1),
    (1, 2),
    (1, 5),
    (1, 8),
    (2, 3),
    (3, 4),
    (5, 6),
    (6, 7),
    (8, 9),
    (9, 10),
    (10, 11),
    (8, 12),
    (12, 13),
    (13, 14),
    (0, 15),
    (0, 16),
    (15, 17),
    (16, 18),
    (17, 18),
    (11, 24),
    (11, 22),
    (22, 23),
    (14, 21),
    (14, 19),
    (19, 20),
]
correct_squat_df = pd.read_csv("../../data/joints/squat.csv")
correct_squat_1_rep_df = correct_squat_df[
    (correct_squat_df["rep"] == 20) & (correct_squat_df["label"] == "correct")
]

In [ ]:
planks_df = pd.read_csv("../../data/train/plank/dct.csv")

In [ ]:
for _, rep in planks_df.groupby(["rep", "label"]):
    print(_, len(rep) / 25)

In [ ]:
correct_squat_1_rep_df

In [ ]:
rep_joint = pd.DataFrame(
    [
        joints.reset_index().loc[1]
        for _, joints in correct_squat_1_rep_df.groupby("frame")
    ]
)

In [ ]:
joint_tensor = torch.tensor(rep_joint[["x", "y", "z"]].values)
dct_2d(joint_tensor).shape

In [ ]:
from scipy.fft import dct, idct

dct(rep_joint["x"].values)

In [ ]:
input = rep_joint[["x", "y", "z"]].values

In [ ]:
input.shape

In [ ]:
from scipy.fftpack import dct, idct


def get_dct_features(input) -> np.ndarray:
    compressed_dct = dct(input, norm="ortho")[:25]
    x_dct = np.zeros(91, dtype=float)

    x_dct[:25] = compressed_dct
    x_idct = idct(x_dct, norm="ortho")
    return x_idct


out = get_dct_features(input[:, 0])

In [ ]:
plt.plot(input[:, 0], label="x")

In [ ]:
plt.plot(out, label="x")

In [ ]:
rep_joint[["x", "y", "z"]].values

In [ ]:
print(out)

In [ ]:
X_LIM = (-1, 1)
Y_LIM = (-1, 1)
Z_LIM = (-1, 1)

ELEV = 0
AZIM = 0

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML


def get_3D_animation(data):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    def update(i):
        ax.clear()
        ax.set_xlim3d(*X_LIM)
        ax.set_ylim3d(*Y_LIM)
        ax.set_zlim3d(*Z_LIM)
        ax.view_init(elev=ELEV, azim=AZIM)

        frame_data = data[data["frame"] == i]
        ax.scatter3D(frame_data["x"], frame_data["y"], frame_data["z"])
        frame_data = frame_data.reset_index()

        for start, stop in openpose_connections:
            ax.plot(
                xs=[frame_data.loc[start]["x"], frame_data.loc[stop]["x"]],
                ys=[frame_data.loc[start]["y"], frame_data.loc[stop]["y"]],
                zs=[frame_data.loc[start]["z"], frame_data.loc[stop]["z"]],
            )

    return animation.FuncAnimation(
        fig, update, frames=pd.unique(data["frame"]), interval=120
    )


ani = get_3D_animation(correct_squat_1_rep_df)
HTML(ani.to_jshtml())